In [1]:
import polars as pl
import requests
from bs4 import BeautifulSoup
import os
import concurrent.futures
import zipfile

### Get href values for download func

In [7]:
url = 'https://dados.rfb.gov.br/CNPJ/'

In [8]:
content = requests.get(url)

In [9]:
soup = BeautifulSoup(content.text,'html')

In [10]:
estabelecimentos = []
for ancor in soup.find_all('a'):
    if 'estabelecimentos' in ancor.text.lower():
        estabelecimentos.append(ancor['href'])
    else:
        pass

In [19]:
estabelecimentos

['Estabelecimentos0.zip',
 'Estabelecimentos1.zip',
 'Estabelecimentos2.zip',
 'Estabelecimentos3.zip',
 'Estabelecimentos4.zip',
 'Estabelecimentos5.zip',
 'Estabelecimentos6.zip',
 'Estabelecimentos7.zip',
 'Estabelecimentos8.zip',
 'Estabelecimentos9.zip']

In [22]:
urls = []
for i in estabelecimentos:
    url = f'https://dados.rfb.gov.br/CNPJ/{i}'
    urls.append(url)

In [23]:
urls

['https://dados.rfb.gov.br/CNPJ/Estabelecimentos0.zip',
 'https://dados.rfb.gov.br/CNPJ/Estabelecimentos1.zip',
 'https://dados.rfb.gov.br/CNPJ/Estabelecimentos2.zip',
 'https://dados.rfb.gov.br/CNPJ/Estabelecimentos3.zip',
 'https://dados.rfb.gov.br/CNPJ/Estabelecimentos4.zip',
 'https://dados.rfb.gov.br/CNPJ/Estabelecimentos5.zip',
 'https://dados.rfb.gov.br/CNPJ/Estabelecimentos6.zip',
 'https://dados.rfb.gov.br/CNPJ/Estabelecimentos7.zip',
 'https://dados.rfb.gov.br/CNPJ/Estabelecimentos8.zip',
 'https://dados.rfb.gov.br/CNPJ/Estabelecimentos9.zip']

In [28]:
filenames = [lower.lower() for lower in estabelecimentos]

### Download function

In [33]:
def download_file(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(f'data/raw/{filename}', 'wb') as file:
            file.write(response.content)
        print(f"File downloaded successfully as '{filename}'")
    else:
        print(f"Failed to download file from '{url}'")

In [34]:
num_threads = min(len(urls),10)

In [35]:
# Download files using multiple threads
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    executor.map(download_file, urls, filenames)

File downloaded successfully as 'estabelecimentos4.zip'
File downloaded successfully as 'estabelecimentos1.zip'
File downloaded successfully as 'estabelecimentos6.zip'
File downloaded successfully as 'estabelecimentos8.zip'
File downloaded successfully as 'estabelecimentos7.zip'
File downloaded successfully as 'estabelecimentos2.zip'
File downloaded successfully as 'estabelecimentos9.zip'
File downloaded successfully as 'estabelecimentos5.zip'
File downloaded successfully as 'estabelecimentos3.zip'
File downloaded successfully as 'estabelecimentos0.zip'


### Extract files

In [37]:
files = os.listdir('data/raw')

In [42]:
files[0]

'estabelecimentos0.zip'

In [46]:
files[0].split('.')[0]

'estabelecimentos0'

In [47]:
for file in files:
    if file.endswith('.zip'):
        file_name = file.split('.')[0]
        file_path = os.path.join('data/raw',file)
        with zipfile.ZipFile(file_path,'r') as zip_ref:
            zip_ref.extractall(f'data/raw/extract/{file_name}')
        print('Done')

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


### Data exploration

In [2]:
pl.Config(tbl_width_chars=1000)
pl.Config.set_tbl_width_chars(None)
pl.Config.set_fmt_str_lengths(1000)
pl.Config(fmt_str_lengths=1000)

In [3]:
emp_file = os.listdir('data/raw/extract/estabelecimentos0/')

In [4]:
new_columns = [
    'cnpj_basic',
    'cnpj_order',
    'cnpj_dv',
    'identificador_matriz',
    'nome_fantasia',
    'situacao_cadastral',
    'data_situacao_cadastral',
    'motivo_situacao_cadastral',
    'nome_cidade_exterior',
    'pais',
    'data_inicio_atividade',
    'cnae_principal',
    'cnae_secundario',
    'tipo_logradouro',
    'logradouro',
    'numero',
    'complemento',
    'bairro',
    'cep',
    'uf',
    'municipio',
    'ddd_1',
    'telefone_1',
    'ddd_2',
    'telefone_2',
    'ddd_fax',
    'fax',
    'email',
    'situacao_especial',
    'data_situacao_especial'
]

In [5]:
dtypes = {
    'cnpj_basic':pl.String,
    'cnpj_order':pl.String,
    'cnpj_dv':pl.String,
    'data_situacao_cadastral':pl.String,
    'identificador_matriz':pl.Int8,
    'situacao_cadastral':pl.Int8,
    'data_inicio_atividade':pl.String,
    'motivo_situacao_cadastral': pl.Int16,
    'pais': pl.Int32,
    'cnae_principal': pl.String,
    'cep': pl.String,
    'uf': pl.Categorical,
    'municipio': pl.Int32,
    'situacao_especial': pl.Categorical
}

In [6]:
df_emp = pl.read_csv(F'data/raw/extract/estabelecimentos0/{emp_file[0]}', encoding = 'latin-1', separator = ';', has_header=False , new_columns = new_columns, dtypes=dtypes)

In [7]:
df_emp.head(5)

cnpj_basic,cnpj_order,cnpj_dv,identificador_matriz,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,pais,data_inicio_atividade,cnae_principal,cnae_secundario,tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_fax,fax,email,situacao_especial,data_situacao_especial
str,str,str,i8,str,i8,str,i16,str,i32,str,str,str,str,str,str,str,str,str,cat,i32,str,str,str,str,str,str,str,cat,str
"""63950158""","""0001""","""42""",1,"""CINTHYA""",8,"""20210728""",1,"""""",null,"""19900720""","""1094500""","""""","""RUA""","""MANUEL GARCIA""","""354""","""""","""VILA BARUEL""","""02523040""","""SP""",7107,"""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""26711226""","""0001""","""94""",1,"""AGROPASTORIL REUNIDAS BAHIA""",8,"""19940831""",9,"""""",null,"""19900821""","""4774100""","""""","""RODOVIA""","""MUNIC ILHA DO BANANAL""","""SN""","""ESQUERDA""","""ZONA RURAL""","""76590000""","""GO""",9601,"""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""63950216""","""0001""","""38""",1,"""""",8,"""20081231""",71,"""""",null,"""19900719""","""8888888""","""""","""RUA""","""GAL JERONIMO FURTADO""","""277""","""""","""JARDIM CABUCU""","""02237000""","""SP""",7107,"""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""63950224""","""0001""","""84""",1,"""""",8,"""20080414""",1,"""""",null,"""19900720""","""5611203""","""""","""RUA""","""MANOEL CAETANO DE GODOY""","""500""","""""","""JARDIM UBIRAMA""","""18683540""","""SP""",6637,"""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""63950364""","""0001""","""52""",1,"""""",8,"""19920527""",1,"""""",null,"""19900720""","""8888888""","""""","""AVENIDA""","""CEL JOSE PIRES ANDRADE""","""101""","""""","""VILA VERA""","""08210040""","""SP""",7107,"""""","""""","""""","""""","""""","""""","""""","""""",""""""


In [8]:
df_emp.estimated_size("mb")

2453.304832458496

In [9]:
# Change it to pl.Categorical

In [10]:
df_emp.filter(pl.col('situacao_especial') != "")['situacao_especial'].unique()

situacao_especial
cat
"""ESPOLIO EV 407"""
"""FALIDO"""
"""EM LIQUIDACAO"""
"""RECUPERACAO JUDICIAL"""


### Change date types

In [ ]:
# data_situacao_cadastral / data_inicio_atividade / data_situacao_especial

In [11]:
df_emp = df_emp.with_columns(
    pl.col('data_situacao_cadastral').str.strptime(pl.Date,'%Y %m %d',strict=False).cast(pl.Date),
    pl.col('data_inicio_atividade').str.strptime(pl.Date,'%Y %m %d',strict=False).cast(pl.Date),
    pl.col('data_situacao_especial').str.strptime(pl.Date,'%Y %m %d',strict=False).cast(pl.Date)
)

In [9]:
df_emp.head(5)

cnpj_basic,cnpj_order,cnpj_dv,identificador_matriz,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,pais,data_inicio_atividade,cnae_principal,cnae_secundario,tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_fax,fax,email,situacao_especial,data_situacao_especial
str,str,str,i8,str,i8,date,i16,str,i32,date,str,str,str,str,str,str,str,str,cat,i32,str,str,str,str,str,str,str,cat,date
"""63950158""","""0001""","""42""",1,"""CINTHYA""",8,2021-07-28,1,"""""",null,1990-07-20,"""1094500""","""""","""RUA""","""MANUEL GARCIA""","""354""","""""","""VILA BARUEL""","""02523040""","""SP""",7107,"""""","""""","""""","""""","""""","""""","""""","""""",null
"""26711226""","""0001""","""94""",1,"""AGROPASTORIL REUNIDAS BAHIA""",8,1994-08-31,9,"""""",null,1990-08-21,"""4774100""","""""","""RODOVIA""","""MUNIC ILHA DO BANANAL""","""SN""","""ESQUERDA""","""ZONA RURAL""","""76590000""","""GO""",9601,"""""","""""","""""","""""","""""","""""","""""","""""",null
"""63950216""","""0001""","""38""",1,"""""",8,2008-12-31,71,"""""",null,1990-07-19,"""8888888""","""""","""RUA""","""GAL JERONIMO FURTADO""","""277""","""""","""JARDIM CABUCU""","""02237000""","""SP""",7107,"""""","""""","""""","""""","""""","""""","""""","""""",null
"""63950224""","""0001""","""84""",1,"""""",8,2008-04-14,1,"""""",null,1990-07-20,"""5611203""","""""","""RUA""","""MANOEL CAETANO DE GODOY""","""500""","""""","""JARDIM UBIRAMA""","""18683540""","""SP""",6637,"""""","""""","""""","""""","""""","""""","""""","""""",null
"""63950364""","""0001""","""52""",1,"""""",8,1992-05-27,1,"""""",null,1990-07-20,"""8888888""","""""","""AVENIDA""","""CEL JOSE PIRES ANDRADE""","""101""","""""","""VILA VERA""","""08210040""","""SP""",7107,"""""","""""","""""","""""","""""","""""","""""","""""",null


### Concating cnpj to a single column and logradouro / change cnae secundario type

In [12]:
df_emp = df_emp.with_columns(
    pl.concat_str(
        [
            pl.col("cnpj_basic"),
            pl.col("cnpj_order"),
            pl.col("cnpj_dv"),
        ]
    ).alias("cnpj_completo"),
    pl.concat_str(
        [
            pl.col("tipo_logradouro"),
            pl.col("logradouro")
        ],
        separator = " ",
    ).alias("logradouro_completo"),
    pl.col('cnae_secundario').cast(pl.List(pl.Utf8))
)

In [13]:
df_emp = df_emp.drop(['cnpj_basic','cnpj_order','cnpj_dv','tipo_logradouro','logradouro'])

In [14]:
df_emp.head(2)

identificador_matriz,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,pais,data_inicio_atividade,cnae_principal,cnae_secundario,numero,complemento,bairro,cep,uf,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_fax,fax,email,situacao_especial,data_situacao_especial,cnpj_completo,logradouro_completo
i8,str,i8,date,i16,str,i32,date,str,list[str],str,str,str,str,cat,i32,str,str,str,str,str,str,str,cat,date,str,str
1,"""CINTHYA""",8,2021-07-28,1,"""""",null,1990-07-20,"""1094500""","[""""]","""354""","""""","""VILA BARUEL""","""02523040""","""SP""",7107,"""""","""""","""""","""""","""""","""""","""""","""""",null,"""63950158000142""","""RUA MANUEL GARCIA"""
1,"""AGROPASTORIL REUNIDAS BAHIA""",8,1994-08-31,9,"""""",null,1990-08-21,"""4774100""","[""""]","""SN""","""ESQUERDA""","""ZONA RURAL""","""76590000""","""GO""",9601,"""""","""""","""""","""""","""""","""""","""""","""""",null,"""26711226000194""","""RODOVIA MUNIC ILHA DO BANANAL"""


#### Re ordering

In [15]:
df_emp = df_emp.select(['cnpj_completo'] + [col for col in df_emp.columns if col not in ['cnpj_completo']])

In [16]:
df_emp.head(5)

cnpj_completo,identificador_matriz,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,pais,data_inicio_atividade,cnae_principal,cnae_secundario,numero,complemento,bairro,cep,uf,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_fax,fax,email,situacao_especial,data_situacao_especial,logradouro_completo
str,i8,str,i8,date,i16,str,i32,date,str,list[str],str,str,str,str,cat,i32,str,str,str,str,str,str,str,cat,date,str
"""63950158000142""",1,"""CINTHYA""",8,2021-07-28,1,"""""",null,1990-07-20,"""1094500""","[""""]","""354""","""""","""VILA BARUEL""","""02523040""","""SP""",7107,"""""","""""","""""","""""","""""","""""","""""","""""",null,"""RUA MANUEL GARCIA"""
"""26711226000194""",1,"""AGROPASTORIL REUNIDAS BAHIA""",8,1994-08-31,9,"""""",null,1990-08-21,"""4774100""","[""""]","""SN""","""ESQUERDA""","""ZONA RURAL""","""76590000""","""GO""",9601,"""""","""""","""""","""""","""""","""""","""""","""""",null,"""RODOVIA MUNIC ILHA DO BANANAL"""
"""63950216000138""",1,"""""",8,2008-12-31,71,"""""",null,1990-07-19,"""8888888""","[""""]","""277""","""""","""JARDIM CABUCU""","""02237000""","""SP""",7107,"""""","""""","""""","""""","""""","""""","""""","""""",null,"""RUA GAL JERONIMO FURTADO"""
"""63950224000184""",1,"""""",8,2008-04-14,1,"""""",null,1990-07-20,"""5611203""","[""""]","""500""","""""","""JARDIM UBIRAMA""","""18683540""","""SP""",6637,"""""","""""","""""","""""","""""","""""","""""","""""",null,"""RUA MANOEL CAETANO DE GODOY"""
"""63950364000152""",1,"""""",8,1992-05-27,1,"""""",null,1990-07-20,"""8888888""","[""""]","""101""","""""","""VILA VERA""","""08210040""","""SP""",7107,"""""","""""","""""","""""","""""","""""","""""","""""",null,"""AVENIDA CEL JOSE PIRES ANDRADE"""


In [17]:
df_emp.estimated_size("gb")

2.4788410095497966